In [1]:
import logging

import hydra

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [2]:
with hydra.initialize(version_base=None, config_path="./config"):
    cfg = hydra.compose(config_name="config", overrides={"dataset.generate_csv=True"})
dataset = hydra.utils.instantiate(cfg.dataset)

Processing ECG / EDA / Label: 100%|██████████| 20/20 [00:00<00:00, 37.63it/s]


In [3]:
dataset.total_df

,Numb,Wav,Unnamed: 2,ECG,Unnamed: 4,E4-EDA,Unnamed: 6,E4-TEMP,Unnamed: 8,Segment ID,Total Evaluation,Unnamed: 11,Unnamed: 12
1,1.0,7.152,15.476,-0.03253542,0.05061065,4.408951,4.651463,34.66,34.59,Sess01_script01_M001,surprise,1.7,4
2,2.0,15.476,19.786,-0.02892037,-0.2566683,0.145914,0.143354,30.69,30.71,Sess01_script01_F001,fear,1.5,3.6
3,3.0,19.786,26.493,-0.4735711,-0.2928188,4.478828,4.746299,34.61,34.55,Sess01_script01_M002,angry,1.3,4.3
4,4.0,26.493,28.943,-0.2928188,2.653444,4.746299,4.579696,34.55,34.53,Sess01_script01_M003,angry,1.5,3.8
5,5.0,28.943,32.167,0.155447,0.02169028,0.134394,0.125435,30.69,30.71,Sess01_script01_F002,fear,1.6,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,513.0,243.997,251.766,-0.03615047,0.003615046,0.550024,0.534646,31.27,31.25,Sess20_impro04_M018,sad,2.3,2.1
514,514.0,251.766,263.213,-0.2060577,-0.06507084,0.216838,0.214275,29.77,29.77,Sess20_impro04_F023,sad,2.5,2.3
515,515.0,263.213,277.349,-0.09037616,0.5277967,0.519269,0.502609,31.31,31.33,Sess20_impro04_M019,sad,2.1,1.8
516,516.0,277.349,290.344,0.01807523,1.200195,0.207868,0.204023,29.73,29.73,Sess20_impro04_F024,happy,3.9,3.2


In [19]:
import pandas as pd

s = pd.read_csv("data/KEMDy19/annotation/Session01_M_res.csv")

In [23]:
s.columns

Index(['Numb', 'Wav', 'Unnamed: 2', 'ECG', 'Unnamed: 4', 'E4-EDA',
       'Unnamed: 6', 'E4-TEMP', 'Unnamed: 8', 'Segment ID', 'Total Evaluation',
       'Unnamed: 11', 'Unnamed: 12', 'Eval01F', 'Unnamed: 14', 'Unnamed: 15',
       'Eval02M', 'Unnamed: 17', 'Unnamed: 18', 'Eval03F', 'Unnamed: 20',
       'Unnamed: 21', 'Eval04F', 'Unnamed: 23', 'Unnamed: 24', 'Eval05M',
       'Unnamed: 26', 'Unnamed: 27', 'Eval06M', 'Unnamed: 29', 'Unnamed: 30',
       'Eval07M', 'Unnamed: 32', 'Unnamed: 33', 'Eval08F', 'Unnamed: 35',
       'Unnamed: 36', 'Eval09M', 'Unnamed: 38', 'Unnamed: 39', 'Eval10F',
       'Unnamed: 41', 'Unnamed: 42'],
      dtype='object')

In [24]:
s["Unnamed: 11"]

0      Valence
1          1.7
2          1.6
3          1.3
4          1.5
        ...   
477        3.6
478        3.4
479        3.5
480        4.2
481        4.4
Name: Unnamed: 11, Length: 482, dtype: object

In [ ]:
def segmentid_parser(segmendID: str):
    